In [1]:
from fastai.text.all import *
from fastai.callback.progress import CSVLogger

In [2]:
import torch
torch.cuda.set_device(2)

In [ ]:
path = untar_data(URLs.IMDB)

In [ ]:
shutil.move(".fastai/data/imdb","../data")

In [4]:
path = "../data/imdb2500"

In [ ]:
# move data into different folders for dataset size (default 25000 train 25000 test)
import os
import shutil
# equivalent to total dataset size of 10000, 20000, 30000, and 40000 respectively
data_sizes = [2500, 5000, 7500, 10000]
status = ['train', 'test']
category = ['pos', 'neg']
for num in data_sizes:
    for t in status:
        for name in category:
            src = f"../data/imdb/{t}/{name}"
            src_files = os.listdir(src)
            dest = f"../data/imdb{num}/{t}/{name}"
            i = 0
            for file_name in src_files:
                full_file_name = os.path.join(src, file_name)
                if os.path.isfile(full_file_name):
                    if (not os.path.isdir(dest)):
                        os.makedirs(dest)
                    shutil.copy(full_file_name, dest)
                i += 1
                if not (i < num):
                    break
            print (len([name for name in os.listdir(dest) if os.path.isfile(os.path.join(dest, name))]))

In [ ]:
dls = DataBlock(
    blocks=(TextBlock.from_folder(path, max_vocab=40000),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
)

In [ ]:
dls = dls.dataloaders(path)
dls.show_batch()

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy, cbs=CSVLogger(append=True))

In [ ]:
learn.fit_one_cycle(1, 1e-2)
learn.remove_cb(CSVLogger)
learn.export('1ephoch.pkl')

In [ ]:
new_learn = load_learner('models/1ephoch.pkl')
sentence = "I love this"
new_learn.predict(sentence)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(6, 1e-4)

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(6, 1e-4)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-4)

In [ ]:
learn.export('23ephoch')

In [54]:
var = learn.show_results(max_n=10, shuffle=True)

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , is n't much different at all from the previous games ( excluding xxmaj tony xxmaj hawk 3 ) . xxmaj the only thing new that is featured in xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , is the new selection of levels , and tweaked out graphics . xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x offers a new career mode , and that is the 2x career . xxmaj the 2x career is basically xxmaj tony xxmaj hawk 1 career , because there is only about five challenges per level . xxmaj if you missed xxmaj tony xxmaj hawk 1 and 2 , i suggest that you buy xxmaj tony xxmaj hawk 's xxmaj pro xxmaj skater 2x , but if you have played the first two games , you should still",pos,pos
2,"xxbos xxmaj based on the excellent novel , xxmaj watchers by xxmaj dean xxmaj koontz , is this extremely awful motion picture that probably should n't be viewed by anyone . xxmaj not since "" the xxmaj running xxmaj man "" have i seen a book butchered so far beyond recognition . xxmaj the difference , however , is that "" the xxmaj running xxmaj man "" film was still enjoyable as an amusing action film laden down a million catch phrases . xxmaj this film ▁ xxmaj nope , nothing remotely amusing . xxmaj in fact , if you love the book , as i do , you 'll hate this bastardization even more . \n\n * * xxunk xxup spoilers * * xxmaj xxunk , xxmaj i 'm basically going to tell you the story here , almost in it 's entirety . xxmaj why ? xxmaj because",neg,neg
3,"xxbos xxmaj hearkening back to those "" good xxmaj old xxmaj days "" of 1971 , we can vividly recall when we were treated with a whole xxmaj season of xxmaj charles xxmaj chaplin at the xxmaj cinema . xxmaj that 's what the promotional guy called it when we saw him on somebody 's old talk show . ( we ca n't recall just whose it was ; either xxup merv xxup griffin or xxup woody xxup woodbury , one or the other ! ) xxmaj the guest talked about xxmaj sir xxmaj charles ' career and how his films had been out of circulation ever since the 1952 exclusion of the former "" little xxmaj tramp ' from xxmaj los xxmaj xxunk xxmaj xxunk on the grounds of his being an "" undesirable xxmaj alien "" . ( no xxmaj schultz , he 's xxup not from another",pos,pos
4,"xxbos "" empire xxmaj strikes xxmaj back "" director xxmaj irvin xxmaj kershner 's "" never xxmaj say xxmaj never xxmaj again , "" a remake of the 1965 xxmaj james xxmaj bond movie "" thunderball , "" does n't surpasses the xxmaj terence xxmaj young original , but this non - harry xxmaj saltzman & xxmaj albert xxup r. xxmaj broccoli film is well worth watching if you call yourself a 007 aficionado . xxmaj nevertheless , despite its shortage of clever gadgets and the lack of a vibrant musical score , "" never xxmaj say xxmaj never xxmaj again "" rates as an above - average , suspenseful doomsday thriller with top - flight performances by a seasoned cast including xxmaj sean xxmaj connery , xxmaj kim xxmaj basinger , xxmaj klaus xxmaj maria xxmaj brandauer , xxmaj max xxmaj von xxmaj sydow , xxmaj barbara xxmaj carrera",pos,pos
5,"xxbos xxmaj during an eight - year stretch of the 1980s when xxmaj charles xxmaj bronson made nine films , only one was released by a company o